In [1]:
import os
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path

import matplotlib.pyplot as plt

from lhotse import CutSet, Fbank, LilcomFilesWriter
from lhotse.augmentation import SoxEffectTransform, RandomValue
from lhotse.dataset import K2SpeechRecognitionDataset
from lhotse.dataset.sampling import SingleCutSampler
from lhotse.recipes.gigaspeech import download_gigaspeech, prepare_gigaspeech

# Settings for paths

In [2]:
root_dir = Path('data')
corpus_dir = root_dir / 'GigaSpeech'
output_dir = root_dir / 'gigaspeech_nb'

# Select data parts

In [3]:
train_set = 'XS'

In [4]:
dataset_parts = (train_set, 'TEST')

# Download the data

In [5]:
password = ''# You need to fill out the Google Form to get the password# https://forms.gle/UuGQAPyscGRrUMLq6


In [6]:
download_gigaspeech(password, corpus_dir, dataset_parts)

# Prepare audio and supervision manifests

In [7]:
num_jobs = os.cpu_count()

In [8]:
gigaspeech_manifests = prepare_gigaspeech(corpus_dir, dataset_parts, output_dir, num_jobs=num_jobs)

# [Optional] Data augmentation

In [9]:
use_data_augmentation = False
augment_fn = SoxEffectTransform(effects=[
   ['reverb', 50, 50, RandomValue(0, 100)],
   ['remix', '-'],  # Merge all channels (reverb changes mono to stereo)
   ['rate', 16000],
]) if use_data_augmentation else None

# Extract features

In [10]:
for partition, manifests in gigaspeech_manifests.items():
    manifest_path = output_dir / f'cuts_{partition}.jsonl.gz'
    if not manifest_path.is_file():
        with ProcessPoolExecutor(num_jobs) as ex:
            cut_set = CutSet.from_manifests(
                recordings=manifests['recordings'],
                supervisions=manifests['supervisions']
            )
            if use_data_augmentation:
                cut_set = cut_set + cut_set.perturb_speed(0.9) + cut_set.perturb_speed(1.1)
            cut_set = cut_set.compute_and_store_features(
                extractor=Fbank(),
                storage_path=f'{output_dir}/feats_{partition}',
                storage_type=LilcomFilesWriter,
                augment_fn=augment_fn,
                num_jobs=num_jobs,
                executor=ex
            )
        gigaspeech_manifests[partition]['cuts'] = cut_set
        cut_set.to_json(manifest_path)
    gigaspeech_manifests[partition] = CutSet.from_jsonl(manifest_path)

# Make PyTorch Dataset

In [11]:
dataset_test = K2SpeechRecognitionDataset(gigaspeech_manifests['TEST'])
dataset_train = K2SpeechRecognitionDataset(gigaspeech_manifests[train_set])

# Illustration of an example

In [12]:
sampler = SingleCutSampler(dataset_test.cuts, shuffle=False, max_cuts=4)
cut_id = next(iter(sampler))[0]
sample = dataset_test[[cut_id]]

In [13]:
seg_id = 1
text = sample['supervisions']['text'][seg_id]
start_frame = int(sample['supervisions']['start_frame'][seg_id])
end_frame = start_frame + int(sample['supervisions']['num_frames'][seg_id]) - 1
feats = sample['inputs'][0][start_frame:end_frame, :]

In [14]:
print('Transcript:', text)
print('Supervisions start frame:', start_frame)
print('Supervisions end frame:', end_frame)
print('Feature matrix:')
plt.matshow(feats.transpose(0, 1).flip(0));

Transcript: WELL THERE'S A NEW SHOW FROM MARKETPLACE AND BRAINS ON THAT WILL HELP ADDRESS JUST THAT <PERIOD>
Supervisions start frame: 486
Supervisions end frame: 1015
Feature matrix:
